# Semantic Kernel Multi-Agent Orchestration Patterns

This notebook explores the five main orchestration patterns available in Semantic Kernel for coordinating multiple AI agents to work together effectively.

## Why Multi-Agent Orchestration?

Traditional single-agent systems are limited in handling complex, multi-faceted tasks. By orchestrating multiple agents with specialized skills, we can create systems that are:
- More robust and adaptive
- Capable of solving real-world problems collaboratively
- Flexible in handling different coordination patterns

## Setup and Imports

In [ ]:
import asyncio
import os
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

# Load environment variables
load_dotenv()

# Azure OpenAI configuration
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

## Common Agent Setup

Let's create some sample agents that we'll use across different orchestration patterns:

In [ ]:
# Create kernel and chat completion service using a helper (aligned with 06-semantic-kernel.ipynb)

def _create_kernel_with_chat_completion():
    """Create a Kernel configured with AzureChatCompletion from environment vars."""
    kernel = Kernel()
    chat_completion_service = AzureChatCompletion(
        deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
        endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    )
    kernel.add_service(chat_completion_service)
    return kernel, chat_completion_service

# Initialize shared kernel and chat service for use across examples
kernel, chat_completion = _create_kernel_with_chat_completion()

# Define specialized agents with descriptions for compatibility with all orchestration patterns
research_agent = ChatCompletionAgent(
    kernel=kernel,
    name="Researcher",
    description="A research specialist that gathers and analyzes information on given topics",
    instructions=(
        "You are a research specialist. Gather and analyze information on given topics. "
        "Provide comprehensive, factual insights."
    ),
)

writer_agent = ChatCompletionAgent(
    kernel=kernel,
    name="Writer",
    description="A professional writer that creates clear, engaging content",
    instructions=(
        "You are a professional writer. Transform research and data into clear, engaging content. "
        "Focus on clarity and readability."
    ),
)

editor_agent = ChatCompletionAgent(
    kernel=kernel,
    name="Editor",
    description="An editor that reviews content for accuracy, grammar, and style",
    instructions=(
        "You are an editor. Review content for accuracy, grammar, and style. "
        "Provide final polishing and quality assurance."
    ),
)

analyst_agent = ChatCompletionAgent(
    kernel=kernel,
    name="Analyst",
    description="A data analyst that identifies patterns and provides insights with supporting data",
    instructions=(
        "You are a data analyst. Analyze information, identify patterns, and provide insights with supporting data."
    ),
)

print("Agents created successfully!")

---

# 1. Sequential Orchestration

**Pattern**: Agents work in a pipeline, each processing the output of the previous agent.

**Use Cases**: 
- Document processing workflows
- Multi-stage content creation
- Data processing pipelines

**Example Flow**: Research → Write → Edit

In [ ]:
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime

# Optional: observe each agent's output as it runs
def agent_response_callback(message):
    try:
        name = getattr(message, "name", "Agent")
        content = getattr(message, "content", "")
        if content:
            print(f"###### Agent Starting {name}\n{content}")
    except Exception:
        pass

async def sequential_example():
    """Demonstrate sequential orchestration pattern"""

    # Create sequential orchestration (matches SK sample style)
    orchestration = SequentialOrchestration(
        members=[research_agent, writer_agent, editor_agent],
        agent_response_callback=agent_response_callback,
    )

    # Start runtime
    runtime = InProcessRuntime()
    runtime.start()

    # Define task
    task = "Create a brief article about the benefits of renewable energy"

    print(f"Task: {task}")
    print("\n=== Sequential Orchestration Flow ===")

    # Invoke orchestration
    result = await orchestration.invoke(task=task, runtime=runtime)
    final_output = await result.get(timeout=20)

    print("\n***** Final Result *****")
    print(final_output)

    # Cleanup
    await runtime.stop_when_idle()

# Run the example
await sequential_example()

---

# 2. Concurrent Orchestration

**Pattern**: Multiple agents work on the same task in parallel, providing diverse perspectives.

**Use Cases**:
- Brainstorming sessions
- Ensemble reasoning
- Voting systems
- Multiple solution generation

**Example Flow**: Research ∥ Analyst ∥ Writer → Aggregated Results

In [ ]:
from semantic_kernel.agents import ConcurrentOrchestration

async def concurrent_example():
    """Demonstrate concurrent orchestration pattern"""
    
    # Create concurrent orchestration
    orchestration = ConcurrentOrchestration(members=[research_agent, analyst_agent, writer_agent])
    
    # Start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    # Define task
    task = "Analyze the pros and cons of remote work. Provide different perspectives."
    
    print(f"Task: {task}")
    print("\n=== Concurrent Orchestration Flow ===")
    
    # Invoke orchestration
    result = await orchestration.invoke(task=task, runtime=runtime)
    outputs = await result.get()
    
    print("\nResults from all agents:")
    for i, output in enumerate(outputs, 1):
        print(f"\nAgent {i} Response:\n{output}")
        print("-" * 50)
    
    # Cleanup
    await runtime.stop_when_idle()

# Run the example
await concurrent_example()

---

# 3. Group Chat Orchestration

**Pattern**: Collaborative conversation among agents with a manager coordinating the flow.

**Use Cases**:
- Team meetings simulation
- Collaborative problem-solving
- Debate and discussion scenarios
- Multi-stakeholder decision making

**Example Flow**: Manager coordinates discussion between specialized agents

In [ ]:
from semantic_kernel.agents import GroupChatOrchestration, RoundRobinGroupChatManager

# Observer function to print the messages from the agents
def agent_response_callback_gc(message):
    """Observer function to print the messages from the agents."""
    try:
        name = getattr(message, "name", "Agent")
        content = getattr(message, "content", "")
        if content:
            print(f"**{name}**\n{content}")
    except Exception:
        pass

async def group_chat_example():
    """Demonstrate group chat orchestration pattern"""
    
    # Add description attribute to existing agents (required for GroupChatOrchestration)
    writer_agent.description = "A professional writer that creates clear, engaging content"
    editor_agent.description = "An editor that reviews content for accuracy, grammar, and style"
    
    # Create group chat orchestration with existing agents
    orchestration = GroupChatOrchestration(
        members=[writer_agent, editor_agent],
        manager=RoundRobinGroupChatManager(max_rounds=5),
        agent_response_callback=agent_response_callback_gc,
    )
    
    # Start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    # Define task
    task = "Create a compelling slogan for a new electric SUV that emphasizes affordability and adventure."
    
    print(f"Task: {task}")
    print("\n=== Group Chat Orchestration ===")
    
    # Invoke orchestration
    result = await orchestration.invoke(task=task, runtime=runtime)
    final_result = await result.get()
    
    print(f"\n***** Final Result *****\n{final_result}")
    
    # Cleanup
    await runtime.stop_when_idle()

# Run the example
await group_chat_example()

---

# 4. Handoff Orchestration

**Pattern**: Agents transfer control based on context or expertise needed.

**Use Cases**:
- Customer support systems
- Expert consultation workflows
- Dynamic task delegation
- Escalation scenarios

**Example Flow**: General Agent → Specialist Agent (based on context)

In [ ]:
from semantic_kernel.agents import HandoffOrchestration, OrchestrationHandoffs
from semantic_kernel.functions import kernel_function

# Simple plugin for demonstration
class SupportPlugin:
    @kernel_function
    def check_order_status(self, order_id: str) -> str:
        """Check the status of an order."""
        return f"Order {order_id} is shipped and will arrive in 2-3 days."
    
    @kernel_function
    def process_refund(self, order_id: str, reason: str) -> str:
        """Process a refund for an order."""
        return f"Refund for order {order_id} has been processed successfully due to: {reason}"

def agent_response_callback_handoff(message):
    """Observer function to print the messages from the agents."""
    try:
        name = getattr(message, "name", "Agent")
        content = getattr(message, "content", "")
        if content:
            print(f"{name}: {content}")
    except Exception:
        pass

async def handoff_example():
    """Demonstrate handoff orchestration pattern"""
    
    # Create specialized agents for handoff with descriptions
    triage_agent = ChatCompletionAgent(
        name="TriageAgent",
        description="A customer support agent that triages issues.",
        instructions="Handle customer requests and route them to the appropriate specialist.",
        kernel=kernel,
    )
    
    order_agent = ChatCompletionAgent(
        name="OrderAgent",
        description="A customer support agent that handles order-related queries.",
        instructions="Handle order status and order-related requests.",
        kernel=kernel,
        plugins=[SupportPlugin()],
    )
    
    refund_agent = ChatCompletionAgent(
        name="RefundAgent",
        description="A customer support agent that handles refunds.",
        instructions="Handle refund requests and process refunds.",
        kernel=kernel,
        plugins=[SupportPlugin()],
    )
    
    # Define handoff relationships with correct method signature
    handoffs = (
        OrchestrationHandoffs()
        .add(
            source_agent=triage_agent.name,
            target_agent=order_agent.name,
            description="Transfer to this agent if the issue is order related",
        )
        .add(
            source_agent=triage_agent.name,
            target_agent=refund_agent.name,
            description="Transfer to this agent if the issue is refund related",
        )
        .add(
            source_agent=order_agent.name,
            target_agent=triage_agent.name,
            description="Transfer back to triage if the issue is not order related",
        )
        .add(
            source_agent=refund_agent.name,
            target_agent=triage_agent.name,
            description="Transfer back to triage if the issue is not refund related",
        )
    )
    
    # Create handoff orchestration
    orchestration = HandoffOrchestration(
        members=[triage_agent, order_agent, refund_agent],
        handoffs=handoffs,
        agent_response_callback=agent_response_callback_handoff,
    )
    
    # Start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    # Define task
    task = "I need help with order 12345 - I want to check its status and possibly get a refund."
    
    print(f"Customer Query: {task}")
    print("\n=== Handoff Orchestration ===")
    
    # Invoke orchestration
    result = await orchestration.invoke(task=task, runtime=runtime)
    resolution = await result.get()
    
    print(f"\n***** Final Resolution *****\n{resolution}")
    
    # Cleanup
    await runtime.stop_when_idle()

# Run the example
await handoff_example()

---

# 5. Magentic Orchestration

**Pattern**: Dynamic, adaptive orchestration with a manager coordinating specialized agents based on evolving context.

**Use Cases**:
- Complex, open-ended tasks
- Research and analysis projects
- Multi-step problem solving
- Dynamic workflow adaptation

**Example Flow**: Manager dynamically selects and coordinates agents based on task progress

In [ ]:
from semantic_kernel.agents import MagenticOrchestration, StandardMagenticManager

def agent_response_callback_magentic(message):
    """Observer function to print the messages from the agents."""
    try:
        name = getattr(message, "name", "Agent")
        content = getattr(message, "content", "")
        if content:
            print(f"**{name}**\n{content}")
    except Exception:
        pass

async def magentic_example():
    """Demonstrate Magentic orchestration pattern"""
    
    # Create magentic orchestration with StandardMagenticManager
    # The StandardMagenticManager uses carefully tuned prompts and requires
    # a chat completion service that supports structured output
    orchestration = MagenticOrchestration(
        members=[research_agent, analyst_agent, writer_agent, editor_agent],
        manager=StandardMagenticManager(chat_completion_service=chat_completion),
        agent_response_callback=agent_response_callback_magentic,
    )
    
    # Start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    # Define complex task
    task = """Create a comprehensive report comparing the energy efficiency and environmental impact 
    of different renewable energy sources (solar, wind, hydroelectric). Include data analysis, 
    cost comparisons, and recommendations for implementation."""
    
    print(f"Complex Task: {task}")
    print("\n=== Magentic Orchestration ===")
    
    # Invoke orchestration
    result = await orchestration.invoke(task=task, runtime=runtime)
    comprehensive_report = await result.get()
    
    print(f"\nComprehensive Report:\n{comprehensive_report}")
    
    # Cleanup
    await runtime.stop_when_idle()

# Run the example
await magentic_example()

---

# Pattern Comparison Summary

| Pattern | Best For | Coordination Style | Output Type |
|---------|----------|-------------------|-------------|
| **Sequential** | Pipelines, step-by-step processing | Linear flow | Single refined result |
| **Concurrent** | Multiple perspectives, brainstorming | Parallel execution | Multiple diverse results |
| **Group Chat** | Collaborative discussions, meetings | Conversational | Consensus or discussion summary |
| **Handoff** | Expert delegation, customer support | Context-based transfer | Specialized resolution |
| **Magentic** | Complex projects, adaptive workflows | Dynamic coordination | Comprehensive solution |

## Key Benefits of Multi-Agent Orchestration

1. **Specialization**: Each agent can focus on their area of expertise
2. **Scalability**: Easy to add new agents or modify workflows
3. **Flexibility**: Different patterns for different use cases
4. **Robustness**: Multiple agents provide redundancy and error checking
5. **Unified Interface**: Same API across all orchestration patterns

## Choosing the Right Pattern

- **Sequential**: When each step depends on the previous one
- **Concurrent**: When you need diverse perspectives or quick parallel processing
- **Group Chat**: When collaborative discussion adds value
- **Handoff**: When different expertise is needed based on context
- **Magentic**: When the task is complex and requires adaptive coordination

## Next Steps

1. Experiment with different patterns for your use cases
2. Customize agent instructions for your domain
3. Implement error handling and retry logic
4. Monitor and optimize agent performance
5. Consider hybrid approaches combining multiple patterns

For more detailed documentation, visit the [Microsoft Learn site for Semantic Kernel Multi-agent Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/).